In [122]:
# config
exampleName="IBM_B3"
exampleDescription="Example ontology for IBM Building 3, Dublin"
testing=False

In [123]:
#!pip install rdflib
#!pip install pydot2
#!pip install pydotplus

In [124]:
import rdflib
from rdflib.namespace import RDFS
from rdflib import URIRef, BNode, Literal
import re
from collections import defaultdict
import numpy as np
import pandas as pd
import pydot as pydot
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
import matplotlib.pyplot as plt

In [125]:
def ns(url):
  url = url.replace("http://buildsys.org/ontologies/brick#", "")
  url = url.replace("http://buildsys.org/ontologies/BuildingSchema#", "")
  return url

BRICK = rdflib.Namespace('http://buildsys.org/ontologies/brick#')
TAGS  = rdflib.Namespace('http://buildsys.org/ontologies/BuildingSchema#')

g = rdflib.Graph()
g.bind('brick', BRICK)
g.bind('bs', TAGS)
result = g.parse('../BuildingSchema/BuildingSchema.ttl', format='n3')
result = g.parse('../BuildingSchema/Brick.owl', format='n3')
print(len(g))

3956


define your own namespace

In [126]:
MyNS = rdflib.Namespace('http://buildsys.org/ontologies/examples/'+exampleName+'#')
g.bind(':', MyNS)

### Load Tag and TagSets from Definition

In [127]:
dfTags=pd.read_csv('../BuildingSchema/Schema Engineering Tags.csv')
schemaTags=set(pd.unique(dfTags.Tag.dropna().ravel()))
len(schemaTags)

139

In [128]:
dfTagSets=pd.read_csv('../BuildingSchema/Schema Engineering TagSets.csv')
schemaTagSets=set()
for ts in pd.unique(dfTagSets.TagSet.dropna().ravel()):
  schemaTagSets.add(ts.replace(' ','_'))
len(schemaTagSets)

486

In [129]:
schemaUsedTags=set()
schemaTagSetTags={}
for ts in schemaTagSets:
  schemaUsedTags.update(ts.split('_'))
  schemaTagSetTags[ts]=set(ts.split('_'))
schemaMissingTags=schemaUsedTags - schemaTags
print("Missing Tags:" + str(len(schemaMissingTags)))
print(schemaMissingTags)

Missing Tags:103
{'Econ-O-Cycle', 'Button', 'Hold', 'grains', 'Tower', 'Overridden', 'Peak', 'AHU', 'Enthalpy', 'Piezoelectric', 'Short', 'Shutdown', 'DualBand', 'Detection', 'Digital', 'Shed', 'HVAC', 'Analog', 'Deionised', 'Effective', 'Lag', 'VFD', 'Fire', 'Radiance', 'Locally', 'Average', 'Stack', 'short', 'Change', 'Integral', 'Humidify', 'To', 'Velocity', 'Activated', 'Dewpoint', 'Sash', 'Highest', 'from', 'by', 'On', 'cycle', 'Stages', 'Lead', 'Environment', 'Compressor', 'Fans', 'Pre', 'Yearly', 'Storage', 'Date', 'Glycool', 'Cycle', 'Thermal', 'Temporary', 'Step', 'Supply', 'Freezer', 'Handler', 'Last', 'Code', 'Wheel', 'Underfloor', 'Meter', 'Generator', 'to', 'Increase', 'Percent', 'CRAC', 'Integration', 'Cold', 'Lowest', 'Perimeter', 'setpoint', 'Push', 'Fume', 'cooling', 'When', 'alarm', 'Face', 'Remotely', 'auto', 'Failed', 'Stage', 'Conductivity', 'Today', 'Monthly', 'Schedule', 'Open', 'Hood', 'Usage', 'Demand', 'Off', 'Even', 'current', 'Preheat', 'Decrease', 'Proporti

### Load TagSets

In [130]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ bs:Tag . }""")
brickTags=set()
for row in qres:
  ts=ns(row['ts'])
  brickTags.add(ns(row['ts']))
len(brickTags)

81

In [131]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ brick:Point . }""")
brickTagSets=set()
for row in qres:
  ts=ns(row['ts'])
  brickTagSets.add(ns(row['ts']))
len(brickTagSets)

323

In [132]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ brick:Location . }""")
loc_tags=set(['Location'])
for row in qres:
  ts=ns(row['ts'])
  loc_tags.add(ns(row['ts']))
len(loc_tags) 

5

In [133]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ brick:Equipment . }""")
len(qres)
asset_tags=set(['Asset'])
for row in qres:
  ts=ns(row['ts'])
  asset_tags.add(ns(row['ts']))
len(asset_tags)

31

In [134]:
brickUsedTags=set()
brickTagSetTags={}
jointTagSetTags={}
for ts in brickTagSets:
  brickUsedTags.update(ts.split('_'))
  brickTagSetTags[ts]=set(ts.split('_'))
  jointTagSetTags[ts]=set(ts.split('_'))
brickMissingTags=brickUsedTags - schemaTags
print("Missing Tags:" + str(len(schemaMissingTags)))
print(schemaMissingTags)

Missing Tags:103
{'Econ-O-Cycle', 'Button', 'Hold', 'grains', 'Tower', 'Overridden', 'Peak', 'AHU', 'Enthalpy', 'Piezoelectric', 'Short', 'Shutdown', 'DualBand', 'Detection', 'Digital', 'Shed', 'HVAC', 'Analog', 'Deionised', 'Effective', 'Lag', 'VFD', 'Fire', 'Radiance', 'Locally', 'Average', 'Stack', 'short', 'Change', 'Integral', 'Humidify', 'To', 'Velocity', 'Activated', 'Dewpoint', 'Sash', 'Highest', 'from', 'by', 'On', 'cycle', 'Stages', 'Lead', 'Environment', 'Compressor', 'Fans', 'Pre', 'Yearly', 'Storage', 'Date', 'Glycool', 'Cycle', 'Thermal', 'Temporary', 'Step', 'Supply', 'Freezer', 'Handler', 'Last', 'Code', 'Wheel', 'Underfloor', 'Meter', 'Generator', 'to', 'Increase', 'Percent', 'CRAC', 'Integration', 'Cold', 'Lowest', 'Perimeter', 'setpoint', 'Push', 'Fume', 'cooling', 'When', 'alarm', 'Face', 'Remotely', 'auto', 'Failed', 'Stage', 'Conductivity', 'Today', 'Monthly', 'Schedule', 'Open', 'Hood', 'Usage', 'Demand', 'Off', 'Even', 'current', 'Preheat', 'Decrease', 'Proporti

In [135]:
brickMissingTagSets=brickTagSets - brickTags
print("Missing Tag Sets:" + str(len(brickMissingTagSets)))
print(brickMissingTagSets)

Missing Tag Sets:322
{'High_Temperature_Hot_Water_Return_Temperature', 'Humidifier_Alarm', 'Supply_Water_Temperature_Setpoint', 'Domestic_Hot_Water_System_Enable', 'Run_Direction_Command', 'Economizer_Differential_Air_Temperature_Setpoint', 'Economizer_Supply_Air_Temperature_Proportional_Band', 'Chilled_Water_Pump_Differential_Pressure_Proportional_Band', 'Supply_Air_Cooling_Demand_High', 'Heating_On_Off_Status', 'Supply_Air_Flow_Demand', 'Outside_Air_Dewpoint_Sensor', 'Domestic_Hot_Water_Supply_Temperature_Setpoint', 'Fault_Indicator', 'Outside_Air_Flow', 'High_Temperature_Alarm', 'Unit_Turned_Off_Remotely', 'Pump_Alarm_Delay', 'Medium_Temperature_Hot_Water_Return_Temperature', 'System_Status', 'Heat_Exchanger_Valve_Command', 'Air_Flow_Loss_Alarm', 'High_Humidity', 'Occupied_Heating_Max_Supply_Air_Flow', 'Outside_Air_Lockout_Temperature_Setpoint', 'Cooling_On_Off_Status', 'Supply_Air_Static_Pressure_Proportional_Band', 'CRAC_Capacity', 'Return_Air_Preheat_Valve_Command', 'Chilled_Wate

In [136]:
def IndivName(name):
  return re.sub(r'\s','_',re.sub(r'[^\d\w\s]', '', name))

### Load Data

In [137]:
df=pd.read_csv('IBM_B3_points.csv')
if testing: df=df.head(20)
df.head()

,Label,TagSet,AssetType,Asset,AssetParent,Site,Building,Wing,BuildingStorey,Room,Zone
0,1F_MID_OPENOFF_CO2,Zone_Air_CO2_Sensor,NaN,NaN,NaN,Dublin,B3,SOR42,FirstFloor,OpenOffice,Middle
1,1F_NRTH_OPENOFF_CO2,Zone_Air_CO2_Sensor,NaN,NaN,NaN,Dublin,B3,SOR42,FirstFloor,OpenOffice,North
2,1F_STH_OPENOFF_CO2,Zone_Air_CO2_Sensor,NaN,NaN,NaN,Dublin,B3,SOR42,FirstFloor,OpenOffice,South
3,421_U10_CLG_VLV,FCU_Cooling_Valve_Percentage_Command,FCU,U10,NaN,Dublin,B3,SOR42,FirstFloor,SOR42_1_U10,NaN
4,421_U10_DAT,FCU_Supply_Air_Temperature_Sensor,FCU,U10,NaN,Dublin,B3,SOR42,FirstFloor,SOR42_1_U10,NaN


Analyze Dataset

In [138]:
dfTags=set()
dfTagSets=set(pd.unique(df.TagSet.dropna().ravel()))
for ts in dfTagSets:
  dfTags.update(ts.split('_'))
len(dfTags)

91

In [139]:
dfMissingTags=dfTags - schemaTags # schemaUsedTags - 
print("Missing Tags from Schema:" + str(len(dfMissingTags)))
print(dfMissingTags)

Missing Tags from Schema:46
{'Factor', 'Extract', 'Supply', 'Footfall', 'Rain', 'Hail', 'Weather', 'Wireless', 'Line', 'Electric', 'AHU', 'Loop', 'Phase', 'Automatic', 'Meter', 'Open', 'Out', 'Blind', 'Light', 'Illumination', 'Lux', 'Socket', 'External', 'Circualation', 'Effective', 'In', 'Frost', 'ThermalWheel', 'Duration', 'Percentage', 'Cold', 'Elevation', 'FCU', 'SRVC', 'Radiation', 'Sun', 'Accumulated', 'Sunblind', 'Presence', 'Trace', 'Incoming', 'Average', 'Facade', 'Instantations', 'Storage', 'PowerUnit'}


In [140]:
dfMissingTags=dfTags - brickTags # schemaUsedTags - 
print("Missing Tags from Bricks:" + str(len(dfMissingTags)))
print(dfMissingTags)

Missing Tags from Bricks:52
{'Factor', 'Extract', 'Rain', 'Footfall', 'Hail', 'Weather', 'Wireless', 'Relative', 'Heat', 'Emergency', 'Line', 'Electric', 'AHU', 'Loop', 'Phase', 'Automatic', 'Meter', 'Open', 'Out', 'Mode', 'Blind', 'Light', 'Lux', 'Socket', 'External', 'Circualation', 'Load', 'Switch', 'In', 'Frost', 'ThermalWheel', 'Duration', 'Percentage', 'Azimuth', 'Cold', 'Elevation', 'Solar', 'FCU', 'SRVC', 'Radiation', 'Run', 'Sun', 'Accumulated', 'Sunblind', 'Presence', 'Trace', 'Incoming', 'Average', 'Facade', 'Instantations', 'Storage', 'PowerUnit'}


Find best matching tag sets for the defined ones

In [141]:
closestMatches={}
fullMatches={}
for ts in pd.unique(df.TagSet.dropna().ravel()):
  if ts in jointTagSetTags:
    closestMatches[ts]=[ts]
    fullMatches[ts]=ts
  else:
    tss=set(ts.split('_'))
    largestSubset=[]
    largestIntersect=0
    for ts2 in jointTagSetTags:
      inters=tss.intersection(jointTagSetTags[ts2])
      if len(inters)>largestIntersect:
        largestIntersect=len(inters)
        largestSubset=[] # reset largestSubset
      if len(inters)==largestIntersect:
        largestSubset.append(ts2) # add to the largest sets
    smalestSubset=largestSubset
    if len(largestSubset)>1:
      smalestDiff=len(tss)
      smalestSubset=[]
      for ts2 in largestSubset:
        diffs=jointTagSetTags[ts2] - tss
        if len(diffs)<smalestDiff:
          smalestDiff=len(diffs)
          smalestSubset=[] # reset largestSubset
        if len(diffs)==smalestDiff:
          smalestSubset.append(ts2) # add to the largest sets
    closestMatches[ts]=smalestSubset
    print(ts+":"+str(smalestSubset))

Zone_Air_CO2_Sensor:['Outside_Air_CO2_Sensor', 'Return_Air_CO2_Sensor']
FCU_Cooling_Valve_Percentage_Command:['Cooling_Valve_Command']
FCU_Supply_Air_Temperature_Sensor:['Supply_Air_Temperature']
FCU_Fan_Speed_Sensor_Command:['Speed_Command']
FCU_Fault_Sensor:['Fault_Indicator', 'Fault_Reset', 'CO2_Sensor', 'Fault_Status', 'Humidity_Sensor', 'Occupancy_Sensor']
FCU_Heating_Valve_Percentage_Command:['Heating_Valve_Command']
FCU_Return_Air_Temperature_Sensor:['Return_Air_Temperature_Sensor']
FCU_Air_Temperature_Setpoint:['Supply_Air_Temperature_Setpoint', 'Mixed_Air_Temperature_Setpoint']
Light_Load_Electric_Meter:['Energy']
Socket_Load_Electric_Meter:['Energy']
Electric_Meter_FCU:['Energy']
Electric_Meter:['Energy']
Electric_Meter_Light:['Energy']
Switch_Emergency_Command:['Emergency_Power_Off_Switch']
Socket_Load_Wireless_Electric_Meter:['Energy']
Socket_Load_Blind_Electric_Meter:['Energy']
Incoming_Water_Meter:['Deionised_Water_Conductivity', 'Deionised_Water_Alarm', 'Water_Loss_Alarm

Manual Mapping based on the results.

In [142]:
myMap={'FCU_Cooling_Valve_Percentage_Command':'Cooling_Valve_Command',
'AHU_Cooling_Valve_Percentage_Command':'Cooling_Valve_Command',
'FCU_Supply_Air_Temperature_Sensor':'Supply_Air_Temperature',
'AHU_Heating_Valve_Percentage_Command':'Heating_Valve_Command',
'AHU_Return_Air_Temperature_Sensor':'Return_Air_Temperature_Sensor',
'AHU_Supply_Air_Temperature_Sensor':'Supply_Air_Temperature',
'AHU_Supply_Air_Temperature_Setpoint':'Supply_Air_Temperature_Setpoint',
'AHU_Zone_Air_Temperature_Sensor':'Zone_Temperature_Sensor',
'FCU_Heating_Valve_Percentage_Command':'Heating_Valve_Command',
'FCU_Return_Air_Temperature_Sensor':'Return_Air_Temperature_Sensor',
'FCU_Fault_Sensor':'Fault_Status',
'FCU_Air_Temperature_Setpoint':'Supply_Air_Temperature_Setpoint',
'Zone_Air_Temperature_Sensor':'Zone_Temperature_Sensor',
'Socket_Load_Blind_Electric_Meter':'Energy',
'Socket_Load_Electric_Meter':'Energy',
'Socket_Load_Frequency_Electric_Meter':'Energy',
'Socket_Load_Line_Voltage_Electric_Meter':'Energy',
'Socket_Load_Phase_Average_PowerUnit_Electric_Meter':'Energy',
'Socket_Load_Phase_Current_Electric_Meter':'Energy',
'Socket_Load_Phase_Electric_Meter':'Energy',
'Socket_Load_Phase_Instantations_PowerUnit_Electric_Meter':'Energy',
'Socket_Load_Phase_Max_PowerUnit_Electric_Meter':'Energy',
'Socket_Load_Power_Factor_Electric_Meter':'Energy',
'Socket_Load_Wireless_Electric_Meter':'Energy',
'Weather_Outside_Air_Temperature_Sensor':'Outside_Air_Temperature_Sensor',
'Weather_Relative_Humidity_Sensor':'Humidity_Sensor',
'Electric_Meter':'Energy',
'Electric_Meter_FCU':'Energy',
'Electric_Meter_Light':'Energy',
'Electric_Meter_SRVC':'Energy',
'FCU_Fan_Speed_Sensor_Command':'Speed_Command',
'AHU_Supply_Fan_Flow_Status_Sensor':'Supply_Fan_Status',
'FCU_Fan_Run_Time_Sensor':'Run_Time',
'Hot_Water_Loop_Return_Temperature_Sensor':'Hot_Water_Return_Temperature',
'Hot_Water_Loop_Supply_Temperature_Sensor':'Hot_Water_Supply_Temperature',
'Light_Load_Electric_Meter':'Energy',
'Light_Load_Phase_Instantations_PowerUnit_Electric_Meter':'Energy',
'Electric_Meter_AHU_Phase':'Energy',
'External_Fan_Variable_Speed_Command':'Speed_Command'}
for ts in myMap: fullMatches[ts]=myMap[ts]

write header

In [143]:
fo = open(exampleName+'.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .
@prefix bs: <http://buildsys.org/ontologies/BuildingSchema#> .
@prefix brick: <http://buildsys.org/ontologies/brick#> .\n""")
fo.write("@prefix : <"+str(MyNS)+"> .\n\n")
fo.write("<"+str(MyNS).replace("#","")+">  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BuildingSchema> , <http://buildsys.org/ontologies/brick> ;\n")
fo.write('\trdfs:comment "'+exampleDescription+'"@en .\n\n')

66

write locations

In [144]:
# create location individuals
locationCols=["Site","Building","Wing","BuildingStorey","Room","Zone"]
locations={}
for col in locationCols:
  for indiv in pd.unique(df[col].dropna().ravel()):
    if indiv!="":
      indivLocName=IndivName(col+"_"+indiv)
      fo.write("\n :"+indivLocName+"  a   brick:"+col+";")
      fo.write('\n\t\t\t rdfs:label "'+indiv+'"@en .\n')
      locations[indiv]=indivLocName
# add relations
for idx in df.index:
  for ci in range(1,len(locationCols)):
    childIndiv =df.loc[idx,locationCols[ci]]
    parentIndiv=df.loc[idx,locationCols[ci-1]]
    if childIndiv in locations and parentIndiv in locations:
      fo.write("\n :"+locations[childIndiv]+"  bs:isPartOf   :"+locations[parentIndiv]+".")
      fo.write("\n :"+locations[parentIndiv]+"  bs:hasPart   :"+locations[childIndiv]+".\n")

write assets

In [145]:
# create asset individuals
assets={}
for idx in df.index:
  assetName = str(df.loc[idx, "Asset"])
  if assetName=="" or assetName=="nan" or assetName in assets: 
    continue
  assetType = df.loc[idx, "AssetType"]
  indivLocName=IndivName(assetType+"_"+assetName)
  fo.write("\n :"+indivLocName+"  a   brick:"+col+";")
  fo.write('\n\t\t\t rdfs:label "'+assetName+'"@en .\n')
  assets[assetName]=indivLocName
  assetParent = str(df.loc[idx, "AssetParent"])
  if assetParent!="" and assetParent!="nan":
    if assetParent not in assets:
      parentLocName=IndivName(assetType+"_"+assetParent)
      fo.write("\n :"+parentLocName+"  a   brick:"+col+";")
      fo.write('\n\t\t\t rdfs:label "'+assetParent+'"@en .\n')
      assets[assetParent]=parentLocName
    fo.write("\n :"+assets[assetName]+"  bs:isPartOf   :"+assets[assetParent]+".")
    fo.write("\n :"+assets[assetParent]+"  bs:hasPart   :"+assets[assetName]+".\n")

analyze tag sets

In [146]:
#create individual for tags
for tag in dfTags:
  if tag in brickTags:
    fo.write("\n :"+tag+"0  a  bs:"+tag+".\n")
  else:
    fo.write("\n :"+tag+"  rdfs:subClassOf  bs:Tag.")
    fo.write("\n :"+tag+"0  a  :"+tag+".\n")

In [147]:
for idx in df.index:
  pointIndivName = IndivName(df.loc[idx, "Label"])
  pointType = df.loc[idx, "TagSet"]
  if pointType in fullMatches:
    fo.write("\n :"+pointIndivName+"  a  bs:Label, bs:TagSet, brick:"+fullMatches[pointType]+";")
  else:
    fo.write("\n :"+pointType+"  rdfs:subClassOf   bs:TagSet.")
    fo.write("\n :"+pointIndivName+"  a   bs:Label, bs:TagSet, :"+pointType+";")
  fo.write('\n \t\t\t rdfs:label "'+df.loc[idx, "Label"]+'"@en ;')
  # write tags
  for tag in set(pointType.split('_')):
    fo.write("\n \t\t\t bs:hasTag   :"+tag+"0;")
  # write location
  loc=df.loc[idx, locationCols].dropna().ravel()[-1]
  fo.write("\n \t\t\t bs:hasLocation :"+locations[loc]+'. \n')
  fo.write("\n :"+locations[loc]+"  bs:hasPoint  :"+pointIndivName+".")
  # write assets
  assetName = str(df.loc[idx, "Asset"])
  if assetName!="" and assetName!="nan":
    fo.write("\n :"+assets[assetName]+"  brick:hasPoint  :"+pointIndivName+".")
    fo.write("\n :"+pointIndivName+"  bs:hasAsset  :"+assets[assetName]+".\n")

In [148]:
fo.close()